# Load data

Apziva project #5: ValueInvestor<br>
2023 09 04

__Summary:__
* This notebook originally loaded an XL file:
    * 15 months.
    * 8 countries.
    * Different operating days among countries.
    * __Problematic:__
        * because of Covid-19.
        * only one year of training data.
* Sandeep Sricharan Mukku recommended to use a much bigger dataset:
    * __Microsoft__
    * One country only.
    * __27 years__ (1986 to 2023).

## TOC: <a class="anchor" id="TOC"></a>
* [Utilities](#Utilities)
* [Installations](#Installations)
* [Loading the data](#LoadingTheData)
* [Streamline data](#StreamlineData)
* [Average orice](#AveragePrice)
* [Save data](#SaveData)

## Utilities <a class="anchor" id="Utilities"></a>
[TOC](#TOC)

In [7]:
# own libraries
import Utilities as u
import MachineLearning as ml

# activate changes in libraries
import importlib
importlib.reload(u)
importlib.reload(ml)

# aliases
from Utilities import TypeChecker as t
from Utilities import PrintAlias as p

## Installations <a class="anchor" id="Installations"></a>
[TOC](#TOC)

In [9]:
# pip install yfinance

## Loading the data <a class="anchor" id="LoadingTheData"></a>
[TOC](#TOC)

__Explanations on options:__
* __Options:__ An option is a __financial contract__ that gives the holder the right, but not the obligation, to buy (call option) or sell (put option) a specific asset, such as a stock, at a predetermined price (strike price) before or on a specified expiration date. Options provide flexibility and can be used for various trading and investment strategies.
* __Calls:__ A call option is a type of stock option that gives the holder the __right to buy__ a specific quantity of a stock at the strike price before or on the expiration date. Call options are typically used when an investor believes the underlying stock's price will rise. By purchasing a call option, the investor can potentially profit from the price increase without actually owning the stock.
* __Puts:__ A put option is the opposite of a call option. It gives the holder the __right to sell__ a specific quantity of a stock at the strike price before or on the expiration date. Put options are often used as a form of insurance or protection against a potential decline in the price of the underlying stock. Investors buy put options when they expect the stock's price to fall, allowing them to sell the stock at a higher strike price.

In [109]:
import yfinance as yf
import pandas as pd
import re

def GetOptionsFromYahoo(strCompany,strExpirationDate,
                        strHistoryPeriod="max", strShareCountStart="2022-01-01",
                        blnVerbose=False):
    '''
    Returns ticker information on a specific company.
    Suppresses most information if blnVerbose is turned off.
    Inspiration: https://pypi.org/project/yfinance/
    
    When     Who What
    07.09.23 dh  Adapted from source given above
    '''
    def PrintTickerDetail(varDetail):
        '''
        Prints ticker details, according to the type of each detail.
        When     Who What
        07.09.23 dh  Created
        '''
        def PartAfterLastDot(strSource):
            lstrParts = strSource.rsplit(".", 1)
            if len(lstrParts) > 1:
                return lstrParts[-1]
            else:
                return strSource
            
        def ForcingDictionaryToDataFrame(dvarSource):
            dstrSerialized = {key: str(value) for key, value in dvarSource.items()}
            dfrSerialized = pd.DataFrame.from_dict(dstrSerialized, orient='index', columns=["Serialized"])
            return dfrSerialized
            
        # variable name
        strVariableName = u.VariableName(varDetail)
        strVariableName = PartAfterLastDot(strVariableName)
        if strVariableName.startswith("dfr"):
            strVariableName = strVariableName[3:]
        strVariableName = strVariableName.upper()
        
        # variable type
        strType = str(type(varDetail))
        strType = re.findall(r"'(.*?)'", strType)[0] # only part between '...'
        strType = PartAfterLastDot(strType)
        
        # display name and type
        p(f"• {strVariableName} ({strType})")
        
        # display contents using optimal format
        # - dataframe if possible 
        if isinstance(varDetail, pd.DataFrame):
            dfrFinal = varDetail
        if isinstance(varDetail, pd.Series):
            dfrFinal = pd.DataFrame(varDetail)
        elif isinstance(varDetail, dict):
            try:
                dfrFinal = pd.DataFrame(varDetail) #  identical entry type
            except:
                dfrFinal = ForcingDictionaryToDataFrame(varDetail) # mixed dictionary data
        elif isinstance(varDetail, list):
            dfrFinal = pd.DataFrame({strVariableName:varDetail})
        elif isinstance(varDetail, tuple):
            dfrFinal = pd.DataFrame({strVariableName:varDetail})
        elif isinstance(varDetail, str):
            p(varDetail)
            p()
            
        if 'dfrFinal' in locals():
            cintExamples = 5
            pd.options.display.max_rows = cintExamples
            u.DisplayDataFrame(dfrFinal)
            pd.reset_option('display.max_rows')
            p(f"Shape: {dfrFinal.shape}")
            p()
    
    # show stopper
    lstrValidOptions = ['1d','5d','1mo','3mo','6mo','1y','2y','5y','10y','ytd','max']
    if not strHistoryPeriod in lstrValidOptions:
        p(f"The history period option '{strHistoryPeriod}' is not valid.")
        p("Valid options:", ", ".join(lstrValidOptions))
        return
    
    # get the company's ticker
    objCompanyTicker = yf.Ticker(strCompany)
    
    # get history
    dfrHistory = objCompanyTicker.history(period=strHistoryPeriod)
    
    # get option chain for specific expiration
    # - data available via: 
    #   - objTickerOptions.calls (dataframe)
    #   - objTickerOptions.puts  (dataframe)
    objTickerOptions = objCompanyTicker.option_chain(strExpirationDate)

    # print all stock info
    if blnVerbose:
        # history
        PrintTickerDetail(objCompanyTicker.info)
        PrintTickerDetail(dfrHistory)
        PrintTickerDetail(objCompanyTicker.history_metadata)
        # actions
        PrintTickerDetail(objCompanyTicker.actions)
        PrintTickerDetail(objCompanyTicker.dividends)
        PrintTickerDetail(objCompanyTicker.splits)
        # ... only for mutual funds & etfs:
        PrintTickerDetail(objCompanyTicker.capital_gains)
        # share count
        sharecount = objCompanyTicker.get_shares_full(start=strShareCountStart, end=None)
        PrintTickerDetail(sharecount)
        # show financials:
        # - income statement
        PrintTickerDetail(objCompanyTicker.income_stmt)
        PrintTickerDetail(objCompanyTicker.quarterly_income_stmt)
        # - balance sheet
        PrintTickerDetail(objCompanyTicker.balance_sheet)
        PrintTickerDetail(objCompanyTicker.quarterly_balance_sheet)
        # - cash flow statement
        PrintTickerDetail(objCompanyTicker.cashflow)
        PrintTickerDetail(objCompanyTicker.quarterly_cashflow)
        # show holders
        PrintTickerDetail(objCompanyTicker.major_holders)
        PrintTickerDetail(objCompanyTicker.institutional_holders)
        PrintTickerDetail(objCompanyTicker.mutualfund_holders)
        # Show future and historic earnings dates, returns at most next 4 quarters and last 8 quarters by default. 
        # Note: If more are needed use objCompanyTicker.get_earnings_dates(limit=XX) with increased limit argument.
        PrintTickerDetail(objCompanyTicker.earnings_dates)
        # show ISIN code - *experimental*
        # ISIN = International Securities Identification Number
        PrintTickerDetail(objCompanyTicker.isin)
        # show options expirations
        PrintTickerDetail(objCompanyTicker.options)
        # show news
        PrintTickerDetail(objCompanyTicker.news)
        # main output
        PrintTickerDetail(objTickerOptions.calls)
        PrintTickerDetail(objTickerOptions.puts)
        # finalize
        p()
        p()
        
    # finalize
    return dfrHistory    

# get history
dstrCompanies = {
    "MSFT":"Microsoft",
    "SYT": "Syngenta"
}
strCompanyAbbreviation = "MSFT"
strHistoryPeriod  = "max"
strExpirationDate = "2023-09-08"
dfrRaw = GetOptionsFromYahoo(strCompanyAbbreviation,strExpirationDate,strHistoryPeriod,blnVerbose=False)
p(f"History of company '{dstrCompanies[strCompanyAbbreviation]}' ({strCompanyAbbreviation})".upper())
dfrRaw

HISTORY OF COMPANY 'MICROSOFT' (MSFT)


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1986-03-13 00:00:00-05:00,0.055004,0.063093,0.055004,0.060396,1031788800,0.0,0.0
1986-03-14 00:00:00-05:00,0.060396,0.063632,0.060396,0.062553,308160000,0.0,0.0
1986-03-17 00:00:00-05:00,0.062553,0.064172,0.062553,0.063632,133171200,0.0,0.0
1986-03-18 00:00:00-05:00,0.063632,0.064172,0.061475,0.062014,67766400,0.0,0.0
1986-03-19 00:00:00-05:00,0.062014,0.062553,0.060396,0.060936,47894400,0.0,0.0
...,...,...,...,...,...,...,...
2023-08-30 00:00:00-04:00,328.670013,329.809998,326.450012,328.790009,15222100,0.0,0.0
2023-08-31 00:00:00-04:00,329.200012,330.910004,326.779999,327.760010,26411000,0.0,0.0
2023-09-01 00:00:00-04:00,331.309998,331.989990,326.779999,328.660004,14931200,0.0,0.0


__Comments__
* There are about 27 years of data for Microsoft.
* The format is somewhat different.
    * No `Price` is given.
    * No `Change %` is given.
    * Only one country.
* The values increase by about 5 orders of magnitude.

## Streamline data <a class="anchor" id="StreamlineData"></a>
[TOC](#TOC)

In [117]:
dfrDataTypesAdjusted = dfrRaw.copy()
lstrColumnsToDrop = ['Volume', 'Dividends', 'Stock Splits']
dfrDataTypesAdjusted = dfrDataTypesAdjusted.drop(columns=lstrColumnsToDrop)
dfrDataTypesAdjusted.index = dfrDataTypesAdjusted.index.date
dfrDataTypesAdjusted.info()
u.DisplayDataFrame(dfrDataTypesAdjusted)

<class 'pandas.core.frame.DataFrame'>
Index: 9447 entries, 1986-03-13 to 2023-09-06
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    9447 non-null   float64
 1   High    9447 non-null   float64
 2   Low     9447 non-null   float64
 3   Close   9447 non-null   float64
dtypes: float64(4)
memory usage: 369.0+ KB


,Open,High,Low,Close
1986-03-13,0.055004,0.063093,0.055004,0.060396
1986-03-14,0.060396,0.063632,0.060396,0.062553
1986-03-17,0.062553,0.064172,0.062553,0.063632
1986-03-18,0.063632,0.064172,0.061475,0.062014
1986-03-19,0.062014,0.062553,0.060396,0.060936
...,...,...,...,...
2023-08-30,328.670013,329.809998,326.450012,328.790009
2023-08-31,329.200012,330.910004,326.779999,327.760010
2023-09-01,331.309998,331.989990,326.779999,328.660004
2023-09-05,329.000000,334.850006,328.660004,333.549988


__Comments__
* The datatypes look reasonable now.

## Average price <a class="anchor" id="AveragePrice"></a>
[TOC](#TOC)

__Price__
* It makes sense to __summarize the 4 price-relevant columns__ into one single column.
* The __median__ of the 4 columns seems to be reasonable:
    * No scale problems (as the prices seem to growth exponentially).
    * Not prone to outliers (`High` and `Low`).

In [119]:
dfrMain = dfrDataTypesAdjusted.copy()
dfrMain['Price'] = dfrMain[['Open', 'High', 'Low', 'Close']].median(axis=1)
p("Column 'Price' contains the median".upper())
dfrMain

COLUMN 'PRICE' CONTAINS THE MEDIAN


,Open,High,Low,Close,Price
1986-03-13,0.055004,0.063093,0.055004,0.060396,0.057700
1986-03-14,0.060396,0.063632,0.060396,0.062553,0.061475
1986-03-17,0.062553,0.064172,0.062553,0.063632,0.063093
1986-03-18,0.063632,0.064172,0.061475,0.062014,0.062823
1986-03-19,0.062014,0.062553,0.060396,0.060936,0.061475
...,...,...,...,...,...
2023-08-30,328.670013,329.809998,326.450012,328.790009,328.730011
2023-08-31,329.200012,330.910004,326.779999,327.760010,328.480011
2023-09-01,331.309998,331.989990,326.779999,328.660004,329.985001
2023-09-05,329.000000,334.850006,328.660004,333.549988,331.274994


## Save data <a class="anchor" id="SaveData"></a>
[TOC](#TOC)

In [ ]:
u.ToDisk(dfrMain,strForcedName="dfrMain",strType="processed")